# O₃ (Ozone) Pollutant Prediction Model

This notebook details the end-to-end process for training a machine learning model to predict O₃ concentrations. The workflow includes:
1. **Setup**: Importing libraries and defining utility functions.
2. **Data Generation**: Creating a dummy dataset for demonstration.
3. **Feature Engineering**: Building time-based, physical, and error-correction features.
4. **Model Training**: Training a LightGBM regressor.
5. **Evaluation**: Assessing model performance on a holdout test set.
6. **Prediction**: Applying the trained model to new, unseen data.

## 1. Setup and Imports

In [ ]:
import os
import warnings
from dataclasses import dataclass
from typing import List

import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Notebook settings
sns.set_context("talk")
sns.set_style("whitegrid")
warnings.filterwarnings("ignore")

## 2. Configuration and Paths

In [ ]:
@dataclass
class TrainingConfig:
    test_size: float = 0.25

config = TrainingConfig()

# Define local file paths
DATAFILE = "site_1_train_data.csv"
UNSEEN_DATA_FILE = "site_1_unseen_input_data.csv"
MODEL_DIR = "models_O3_high_performance"
os.makedirs(MODEL_DIR, exist_ok=True)

## 3. Dummy Data Generation

This cell creates mock `train` and `unseen` data files, allowing the notebook to run from start to finish without external dependencies.

In [ ]:
def create_dummy_data(filename, num_rows):
    """Creates a dummy CSV file with the required schema."""
    dates = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_rows, freq='h'))
    data = {
        'year': dates.year, 'month': dates.month, 'day': dates.day, 'hour': dates.hour,
        'O3_forecast': 50 + np.random.randn(num_rows) * 20 + 20 * np.sin(np.arange(num_rows) * 2 * np.pi / 24),
        'NO2_forecast': 40 + np.random.randn(num_rows) * 15 - 15 * np.sin(np.arange(num_rows) * 2 * np.pi / 24),
        'T_forecast': 25 + np.random.randn(num_rows) * 5,
        'u_forecast': np.random.randn(num_rows) * 2,
        'v_forecast': np.random.randn(num_rows) * 2,
    }
    if 'train' in filename:
        data['O3_target'] = (data['O3_forecast'] + np.random.randn(num_rows) * 5).clip(lower=0)
        data['NO2_target'] = (data['NO2_forecast'] + np.random.randn(num_rows) * 5).clip(lower=0)
    
    df = pd.DataFrame(data)
    df['O3_forecast'] = df['O3_forecast'].clip(lower=0)
    df['NO2_forecast'] = df['NO2_forecast'].clip(lower=0)
    df.to_csv(filename, index=False)
    print(f"Created dummy data file: {filename}")

# Create the files
create_dummy_data(DATAFILE, num_rows=8760) # 1 year of data
create_dummy_data(UNSEEN_DATA_FILE, num_rows=168) # 1 week of data

## 4. Utility Functions

In [ ]:
def ria_score(y_true, y_pred):
    """Calculates the Refined Index of Agreement (RIA)."""
    numerator = np.sum(np.abs(y_true - y_pred))
    denominator = 2 * np.sum(np.abs(y_true - np.mean(y_true)))
    return 1.0 if denominator == 0 else 1 - (numerator / denominator)

def detailed_evaluation(y_true, y_pred, name="Dataset"):
    """Calculates and prints detailed evaluation metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    ria = ria_score(y_true, y_pred)
    print(f"\n=== {name} Metrics ===")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2: {r2:.4f}")
    print(f"RIA: {ria:.4f}")
    print(f"Prediction Range: Min={y_pred.min():.4f}, Max={y_pred.max():.4f}")

def time_aware_split(X, y, timestamps, test_size):
    """Splits time-series data chronologically."""
    split_idx = int(len(X) * (1 - test_size))
    return X[:split_idx], X[split_idx:], y[:split_idx], y[split_idx:], timestamps[:split_idx], timestamps[split_idx:]

## 5. Feature Engineering

In [ ]:
def build_features(df: pd.DataFrame) -> (pd.DataFrame, List[str]):
    df = df.copy()
    if not isinstance(df.index, pd.DatetimeIndex):
        df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
        df = df.set_index('datetime').sort_index()

    # Time and physical features
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    for col in ['hour', 'dayofweek', 'month']:
        df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
        df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df['wind_speed'] = np.sqrt(df['u_forecast']**2 + df['v_forecast']**2)

    # Lag and rolling features
    for col in ['O3_forecast', 'NO2_forecast', 'T_forecast', 'wind_speed']:
        if col in df.columns:
            for L in [1, 2, 3, 6, 12, 24]:
                df[f'{col}_lag{L}'] = df[col].shift(L)
            for W in [6, 12, 24]:
                df[f'{col}_rolling_mean_{W}h'] = df[col].rolling(W, min_periods=1).mean()

    # Error correction for O3
    if 'O3_target' in df.columns:
        df['forecast_error'] = df['O3_forecast'] - df['O3_target']
        for L in [1, 2, 3, 6, 12, 24]:
            df[f'forecast_error_lag{L}'] = df['forecast_error'].shift(L)

    base_cols = [col for col in df.columns if 'target' not in col and 'forecast_error' not in col]
    feature_cols = [col for col in base_cols if col not in ['year', 'day', 'hour', 'month', 'dayofweek', 'u_forecast', 'v_forecast']]
    return df.ffill().bfill().fillna(0), feature_cols

## 6. Model Training Workflow

In [ ]:
# Load and process data
df, feature_cols = build_features(pd.read_csv(DATAFILE))
train_feature_cols = feature_cols + [f'forecast_error_lag{L}' for L in [1, 2, 3, 6, 12, 24]]
print(f"Training O3 model with {len(train_feature_cols)} features.")

# Split data
y = df["O3_target"].ffill().bfill().values
X = df[train_feature_cols].values
X_train, X_test, y_train, y_test, t_train, t_test = time_aware_split(X, y, df.index, config.test_size)

# Scale features
scaler = StandardScaler().fit(X_train)
X_train_s, X_test_s = scaler.transform(X_train), scaler.transform(X_test)
joblib.dump(scaler, os.path.join(MODEL_DIR, "scaler_O3.joblib"))

# Train model
print("\n--- Training O3 Model ---")
model_params = {
    'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2000, 'learning_rate': 0.02,
    'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'num_leaves': 128, 'verbose': -1, 'n_jobs': -1, 'seed': 42
}
model = lgb.LGBMRegressor(**model_params)
model.fit(X_train_s, y_train, eval_set=[(X_test_s, y_test)], eval_metric='rmse', callbacks=[lgb.early_stopping(100)])
joblib.dump(model, os.path.join(MODEL_DIR, "final_model_O3.joblib"))

# Evaluate model
print("\n--- Evaluating O3 Model ---")
final_preds = model.predict(X_test_s)
detailed_evaluation(y_test, final_preds, name="O3 Holdout Test Set")

## 7. Visualization of Results

In [ ]:
def save_plot(filename, title, xlabel, ylabel):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if plt.gca().get_legend_handles_labels()[1]:
        plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_DIR, filename), dpi=150)
    plt.close()

print("\nGenerating O3 evaluation plots...")

# Time-series plot
plt.figure(figsize=(15, 6))
plt.plot(t_test, y_test, label="Actual", alpha=0.8)
plt.plot(t_test, final_preds, label="Predicted", linestyle='--')
save_plot("test_overlay_O3.png", "Actual vs. Predicted O3", "Datetime", r"O3 Concentration ($\mu g/m^3$)")

# Scatter plot
plt.figure(figsize=(7, 7))
plt.scatter(y_test, final_preds, alpha=0.5)
lims = [min(np.nanmin(y_test), np.nanmin(final_preds)), max(np.nanmax(y_test), np.nanmax(final_preds))]
plt.plot(lims, lims, 'k--', alpha=0.75, zorder=0, label="Ideal")
plt.xlim(lims); plt.ylim(lims)
save_plot("test_scatter_O3.png", "Predicted vs. Actual Scatter Plot", "Actual O3", "Predicted O3")

# Forecast vs. Predicted plot
o3_forecast_test = df.loc[t_test, "O3_forecast"].values
plt.figure(figsize=(15, 6))
plt.plot(t_test, o3_forecast_test, label="O3 Forecast", alpha=0.8)
plt.plot(t_test, final_preds, label="O3 Predicted (Corrected)", linestyle='--', color='coral')
save_plot("forecast_vs_predicted_test_O3.png", "O3 Forecast vs. Model Prediction (Test Data)", "Datetime", r"O3 Concentration ($\mu g/m^3$)")

print(f"O3 evaluation plots saved to '{MODEL_DIR}'")

## 8. Prediction on Unseen Data

In [ ]:
if os.path.exists(UNSEEN_DATA_FILE):
    print("\n--- Predicting O3 on unseen data ---")
    udf, unseen_feature_cols = build_features(pd.read_csv(UNSEEN_DATA_FILE))

    X_unseen = udf[unseen_feature_cols].reindex(columns=train_feature_cols).fillna(0).values
    X_unseen_s = scaler.transform(X_unseen)
    final_unseen_preds = model.predict(X_unseen_s)

    results = pd.DataFrame(index=udf.index, data={
        'O3_forecast': udf['O3_forecast'],
        'O3_predicted': final_unseen_preds
    })
    out_path = os.path.join(MODEL_DIR, "unseen_predictions_O3.csv")
    results.to_csv(out_path)
    print(f"Saved unseen O3 predictions -> {out_path}")

    # Plot unseen data results
    plt.figure(figsize=(15, 6))
    plt.plot(results.index, results['O3_forecast'], label="O3 Forecast", alpha=0.8)
    plt.plot(results.index, results['O3_predicted'], label="O3 Predicted (Corrected)", linestyle='--')
    save_plot("forecast_vs_predicted_unseen_O3.png", "O3 Forecast vs. Model Prediction (Unseen Data)", "Datetime", r"O3 Concentration ($\mu g/m^3$)")
    print(f"Unseen data plot saved to '{MODEL_DIR}'")
